In [1]:
from enhanced_lib.exchange import Account,ExchangeCache
from enhanced_lib.exchange.cache import FutureInstance
from enhanced_lib.calculations import workers

account = Account("main_account",'sub_account',base_url='http://localhost:8000')

exchange = ExchangeCache(account,'BTCUSDT')

# await exchange.fetch_latest(False)

# exchange.open_orders.get_tp_orders('long',True)

await account.fetch_config('BTCUSDT')
await exchange.fetch_latest(True)
# exchange.open_orders.get_sl_orders('short',True)
# await account.update_config_fields('BTCUSDT',{'budget':400,'max_size':0.21})
# await account.update_config_resistance('BTCUSDT')
# account.general_config['config']['BTCUSDT']
# exchange.future_instance.config.as_dict()


ImportError: cannot import name 'Account' from 'enhanced_lib.exchange' (/home/beesama/projects/bot-trader/enhanced-lib/python/enhanced_lib/exchange/__init__.py)

In [3]:
future_instance = exchange.future_instance

# future_instance.get_trading_zones('long')
# future_instance.config.long_liquidation_price
# future_instance.config.as_dict()
# zones = future_instance.config.get_trading_zones('short')
# zones
# stop = future_instance.config.long_liquidation_price
# stop 
# app_config = future_instance.config.app_config
# app_config.entry = 44779.3
# app_config.stop = stop
# # app_config.raw = True
# # workers.determine_optimum_reward(app_config)
# # future_instance.config.determine_optimum_risk('long',{
# #     'entry': 44779.3,
# #     'stop': stop
# # })
# exchange.maximum_sell_size
exchange.get_zone_for_position('long')
# exchange.get_zone_for_position('long')
# exchange.get_next_tradable_zone('long')
# exchange.config_params_for_future_trades2('long',True)

waiting for all trades to be processed []
waiting for all trades to be processed []
size for risk 6 0.023
size for risk 6 0.025
size for risk 7 0.028
waiting for all trades to be processed []
size for risk 7 0.031
size for risk 9 0.035
waiting for all trades to be processed []
size for risk 9 0.039
size for risk 12 0.055
size for risk 12 0.05
waiting for all trades to be processed []
size for risk 16 0.075
size for risk 16 0.068
waiting for all trades to be processed []
size for risk 21 0.103
size for risk 21 0.094
waiting for all trades to be processed []
size for risk 27 0.135
size for risk 27 0.122
waiting for all trades to be processed []
size for risk 34 0.174
waiting for all trades to be processed []
size for risk 34 0.159
size for risk 42 0.219
waiting for all trades to be processed []
size for risk 42 0.202
size for risk 51 0.271
waiting for all trades to be processed []
size for risk 51 0.25
waiting for all trades to be processed []
size for risk 61 0.332
size for risk 61 0.30

In [7]:
import concurrent.futures
import time

def worker_function(x,timer=1):
    # Your worker function logic goes here
    result = x * x
    time.sleep(timer)
    print(f"Processed {x}, result: {result}")
    return result

def callback(future):
    try:
        result = future.result()
        print(f"Callback: Task completed. Result: {result}")
        # Perform a specific task with the result
    except Exception as e:
        print(f"Callback: Task raised an exception: {e}")

def main():
    data = [(1,2), (2,1), (3,3), (4,2), (5,3)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(worker_function, *value) for value in data]

        # Add a callback function to each future
        for future in futures:
            future.add_done_callback(callback)
        print('continue processing')

    print("All tasks submitted. The program will exit once all callbacks are executed.")

main()


continue processing
Processed 2, result: 4
Callback: Task completed. Result: 4
Processed 4, result: 16
Callback: Task completed. Result: 16
Processed 1, result: 1
Callback: Task completed. Result: 1
Processed 3, result: 9
Callback: Task completed. Result: 9
Processed 5, result: 25
Callback: Task completed. Result: 25
All tasks submitted. The program will exit once all callbacks are executed.
